In [64]:
import pandas as pd
import numpy as np

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
import csv
import os
from sqlite3 import connect, Error


get_query = """
    SELECT restaurant_orderitem.quantity, restaurant_order.datetime, restaurant_product.name, restaurant_product.price FROM restaurant_orderitem
    INNER JOIN restaurant_order ON restaurant_order.id = restaurant_orderitem.order_id
    INNER JOIN restaurant_product ON restaurant_product.id == restaurant_orderitem.product_id;
"""
db_filename = "db.sqlite3"
csv_filename = "orders_data.csv"

try:
    conn = connect(os.path.join(os.pardir, db_filename))
    with conn:
        cursor = conn.cursor()
        cursor.execute(get_query)
        headers = [col[0] for col in cursor.description]
        data = cursor.fetchall()
except Error as error:
    print(error)

csv_path = os.path.join(os.pardir, csv_filename)
with open(csv_path, "w") as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(headers)
    csvwriter.writerows(data)


df = pd.read_csv(csv_path)
df.info()


# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
top_popular = df.groupby("name").quantity.sum().sort_values(ascending=False).head(10)
top_popular

top_popular.plot.pie(labels=top_popular.index, autopct="%1.1f%%", title="Top 10 sold by quantity", figsize=(8, 8))


# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
top_by_money = df.assign(total_price=df.quantity*df.price).groupby("name", as_index=False).total_price.sum().sort_values("total_price", ascending=False).head(10)
top_by_money.plot.pie(y="total_price", labels=top_by_money.name, autopct="%1.1f%%", title="Top 10 sold by value", figsize=(8, 8))


# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["datetime"] = df["datetime"].astype("datetime64[ns]")
top_hour = df.assign(order_hour=df.quantity*df.price, hour=df.datetime.dt.hour).groupby("hour").order_hour.sum()
top_hour.plot.bar(y="order_hour", figsize=(13, 10), title="Profit by order hour")


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
top_weekday = df.assign(order_weekday=df.quantity*df.price, day_of_week=df.datetime.dt.day_name()).groupby("day_of_week").order_weekday.sum()
top_weekday.plot.bar(y="order_weekday", figsize=(13, 10), title="Profit by order weekday")
